In [18]:
from pyspark.sql import SparkSession, Row
import re
from operator import add
from pyspark.sql.functions import explode_outer

In [21]:
spark = SparkSession.builder.appName("Testing").getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
rdd_data = spark.read.json('processed_data.json')
rdd_data.take(1)

[Row(language='zh', neighbor_urls=['http://idinfo.zjamr.zj.gov.cn', 'http://www.beian.miit.gov.cn', 'http://www.0575ls.cn', 'http://bdimg.share.baidu.com', 'https://hm.baidu.com', 'https://webscan.qianxin.com', 'http://www.beian.gov.cn', 'http://img.ksbbs.com', 'http://g.0575ls.cn'], parsed_text='登录   注册会员     购房  楼盘大全 地图找房 购房36计 看房日记   资讯  新\u2003闻 专\u2003题 土\u2003地 成交数据 地产招聘 业主论坛   服务  看\u2002房\u2002团  印象楼盘 置业大讲堂    城市  城市规划 销焦精英 地产名人堂 地产下午茶               搜楼盘 搜新闻         区域越城区柯桥区诸暨上虞嵊州新昌     面积 70㎡以下 70-90㎡ 90-120㎡ 120-150㎡ 150-200㎡ 200-300㎡ 300㎡以上     价格 12000元/㎡以下 12000-15000元/㎡ 15000-18000元/㎡ 18000-21000元/㎡ 21000-25000元/㎡ 25000元/㎡以上        高级搜索         高级搜索           绍兴楼市 » 楼市资讯 » 全国资讯 杭州：共有产权保障住房“同权”商品住房  2021-12-03记者：  来源：中国网地产 分享到：   杭州市1日对外发布《杭州市共有产权保障住房管理办法》，其中明确，共有产权保障住房面向符合条件的市区户籍和稳定就业的非市区户籍家庭供应，购买共有产权保障住房享有与购买商品住房同等的公共服务权益。 根据办法，市区户籍家庭要在符合限购政策前提下同时满足户籍、住房等相关条件，非市区户籍家庭需满足居住证、住房以及累计缴纳社保或个税年限等相关条件。 办法明确，共有产权保障住房销售基准价按同地段、同类型商品住房市场价格合理优惠后确定。单套销售价格按照销售基准价及其浮动幅度确定，在售房阶段向社会公布。划拨土

In [ ]:
def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    #parts = re.split(r'\s+', urls)
    parts = urls
    return parts[0], parts[1]


# OLD METHOD
lines_testfile = spark.read.text('testfile.txt').rdd.map(lambda r: r[0])
print(lines_testfile.take(5))


# NEW METHOD
rdd_data = spark.read.json('processed_data.json')
exploded = rdd_data.select('url', explode_outer(rdd_data.neighbor_urls).alias("neighbour_url"))
lines_json = exploded.rdd.map(lambda x: x)
print(lines_json.take(5))


links = lines_json.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
#print(links.take(5))
#links.take(5)

[('http://0575ls.cn/news-52300.htm',
  <pyspark.resultiterable.ResultIterable at 0x7f122df9cc40>),
 ('http://0575ls.cn/pan/list?j=4&qu=0&lpmj=0',
  <pyspark.resultiterable.ResultIterable at 0x7f122df9c760>),
 ('http://08.od.ua/bytovaya_tehnika/stiralnye_mashiny/simens_bosh_magazin_bytovoy_tehniki',
  <pyspark.resultiterable.ResultIterable at 0x7f122df9cbe0>),
 ('http://100lat.pck.pl/fakty/pawel-sapieha/',
  <pyspark.resultiterable.ResultIterable at 0x7f122df9c0a0>),
 ('http://121.100.18.218/tampil_ipk_ikm/ipk_ikm.php',
  <pyspark.resultiterable.ResultIterable at 0x7f122df9c3a0>)]

In [ ]:
exploded.show()


+--------------------+--------------------+
|                 url|       neighbour_url|
+--------------------+--------------------+
|http://0575ls.cn/...|http://idinfo.zja...|
|http://0575ls.cn/...|http://www.beian....|
|http://0575ls.cn/...|http://www.0575ls.cn|
|http://0575ls.cn/...|http://bdimg.shar...|
|http://0575ls.cn/...|https://hm.baidu.com|
|http://0575ls.cn/...|https://webscan.q...|
|http://0575ls.cn/...|http://www.beian....|
|http://0575ls.cn/...|http://img.ksbbs.com|
|http://0575ls.cn/...|  http://g.0575ls.cn|
|http://0575ls.cn/...|http://idinfo.zja...|
|http://0575ls.cn/...|http://www.beian....|
|http://0575ls.cn/...|http://www.0575ls.cn|
|http://0575ls.cn/...|https://webscan.q...|
|http://0575ls.cn/...|https://hm.baidu.com|
|http://0575ls.cn/...|http://www.beian....|
|http://0575ls.cn/...|http://cdn.bootcs...|
|http://0575ls.cn/...|  http://g.0575ls.cn|
|http://08.od.ua/b...|     http://08.od.ua|
|http://08.od.ua/b...|http://top100.ram...|
|http://08.od.ua/b...|http://www

In [ ]:
# Code copied from: https://github.com/apache/spark/blob/master/examples/src/main/python/pagerank.py
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    
    #parts = re.split(r'\s+', urls)
    parts = urls
    return parts[0], parts[1]


def pageRank(file, iters = 10):
    """Naive PageRank algorithm."""
    
    # Initialize the spark context.
    spark = SparkSession\
        .builder\
        .appName("PythonPageRank")\
        .getOrCreate()

    # Loads in input file. It should be in format of:
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     ...

    #lines = spark.read.text(file).rdd.map(lambda r: r[0])
    
    rdd_data = spark.read.json(file)
    exploded = rdd_data.select('url', explode_outer(rdd_data.neighbor_urls).alias("neighbour_url"))
    lines = exploded.rdd.map(lambda x: x)
    # Loads all URLs from input file and initialize their neighbors.
    links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
    #print(links.take(5))
    # Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
    ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))
    #print(ranks.take(5))

    # Calculates and updates URL ranks continuously using PageRank algorithm.
    for iteration in range(int(iters)):
        # Calculates URL contributions to the rank of other URLs.
        contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(
            url_urls_rank[1][0], url_urls_rank[1][1]  # type: ignore[arg-type]
        ))
        # Re-calculates URL ranks based on neighbor contributions.
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
        print(ranks.take(5))
    #print(ranks.take(5))
    # Collects all URL ranks and dump them to console.
    for (link, rank) in ranks.collect():
        print("%s has rank: %s." % (link, rank))

    spark.stop()

#pageRank("testfile.txt")
pageRank('processed_data.json')

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
hadoop_rdd = spark.sparkContext.textFile("hdfs://namenode:9000/crawl-data/warc/processed_data_new.txt")

AttributeError: 'NoneType' object has no attribute 'sc'